# app

> FastAPI app

In [ ]:
# | default_exp api/app

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
import os
from typing import Optional

import pymupdf
import uvicorn
from fastapi import FastAPI, File, HTTPException, UploadFile

from rager.api.lifespan import lifespan
from rager.embed import EmbedRequest, embed_supported_models
from rager.rerank import RerankRequest
from rager.tokens import count_tokens

In [ ]:
# | export
app = FastAPI(lifespan=lifespan)

In [ ]:
# | export


@app.get("/models")
async def models():
    embed_clients = app.state.clients["embed"]
    embed_models = [
        k
        for k, v in embed_supported_models.items()
        if v.init_key in embed_clients.keys()
    ]
    return {"embed": embed_models}

In [ ]:
# | export


@app.post("/embed")
async def embed(req: EmbedRequest):
    if req.model_str not in embed_supported.keys():
        raise HTTPException(status_code=400, detail=f"Unsupported model: {embed.model}")

    model = embed_supported_models[req.model_str]

    if isinstance(req.text, str):
        req.text = [req.text]

    if len(req.text) > model.max_batch:
        raise HTTPException(
            status_code=400,
            detail=f"Batch size {len(req.text)} exceeds model {req.model} max batch {model.max_batch}",
        )

    client = None
    if model.init_key is not None:
        if model.init_key not in app.state.clients.keys():
            raise HTTPException(
                status_code=404, detail=f"Client {model.init_key} not found"
            )
        client = app.state.clients[model.init_key]
        if client is None:
            raise HTTPException(
                status_code=500,
                detail=f"Client '{model.init_key}' is not initialized",
            )

    fn_embed = model.fn_embed(client)

    embeddings = []
    for i, text in enumerate(req.text):
        if count_tokens(text) > model.max_input_size:
            raise HTTPException(
                status_code=400,
                detail=f"Input too long for model {req.model} with max length {model.max_input_size} tokens at index {i}",
            )

        if model.async_:
            embedding = await fn_embed(text)
        else:
            embedding = fn_embed(text)

        embeddings.append(embedding)

    return {"embeddings": embeddings}

In [ ]:
# | export


@app.post("/rerank")
async def rerank():
    return {"rerank": "not implemented"}

In [ ]:
# | export


@app.post("/complete")
async def complete():
    return {"complete": "not implemented"}

In [ ]:
# | export


@app.post("/chat")
async def chat():
    return {"chat": "not implemented"}

In [ ]:
# | export


class UploadRequest:
    file: UploadFile
    chunk_strategy: Optional[str] = None

In [ ]:
# | export


@app.post("/upload/chunk_strategy")
async def create_upload_file(file: UploadFile, chunk_strategy: str | None = None):
    if file.content_type == "text/plain":
        contents = await file.read()
    elif file.content_type == "application/pdf":
        contents = pymupdf4llm.to_markdown(file.file)
    else:
        raise HTTPException(
            status_code=400, detail=f"Unsupported file type: {file.content_type}"
        )

In [ ]:
# | export
# | eval: false
if __name__ == "__main__":
    uvicorn.run(
        "rager.api.app:app",
        host=os.getenv("RAGER_API_HOST", "0.0.0.0"),
        port=os.getenv("RAGER_API_PORT", 8080),
        reload=os.getenv("RAGER_API_RELOAD", False),
    )

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()